In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import paths
import functions_bondjango as bd
import functions_loaders as fl
import functions_plotting as fp
import snakemake_scripts.classify_batch as class_fun
import yaml
import processing_parameters
import datetime

import numpy as np
import pandas as pd
from sklearn import preprocessing
import sklearn.metrics as smet
import sklearn.linear_model as lin
import sklearn.model_selection as mod
import sklearn.svm as svm
import sklearn.multiclass as multi
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.graphics.tsaplots import plot_acf

import h5py
import functions_misc as fm
import random
import scipy.stats as stat
import scipy.signal as signal
import importlib

import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
from bokeh.resources import INLINE
from bokeh import palettes

In [ ]:
# set up the figure config
importlib.reload(fp)
importlib.reload(processing_parameters)
# define the target saving path
save_path = os.path.join(paths.figures_path, 'Regression_prototype')

# define the printing mode
save_mode = True
# define the target document
target_document = 'poster'
# set up the figure theme
fp.set_theme()
# load the label dict
label_dict = processing_parameters.label_dictionary
variable_list = processing_parameters.variable_list
units_dict = processing_parameters.tc_units

In [ ]:
# load the target files

# get the search string
animal = processing_parameters.animal
day = processing_parameters.day
rig = processing_parameters.rig
search_string = 'imaging:doric, slug:%s' % day

# query the database for data to plot
data_all = bd.query_database('analyzed_data', search_string)

input_path = [el['analysis_path'] for el in data_all if ('preproc' in el['slug'] and animal.lower() in el['slug'])]
print(input_path)

In [ ]:
# load the data
data_list = []
meta_list = []
frame_list = []
for idx, el in enumerate(input_path):
    # get the trial timestamp (for frame calculations)
    time_stamp = int(''.join(os.path.basename(el).split('_')[3:6]))
    # also get the trial time signature, used as unique ID
    # time_signature = int(''.join(os.path.basename(el).split('_')[0:6]))

    try:
        temp_data = pd.read_hdf(el, 'matched_calcium')
        # temp_data['trial_id'] = time_signature
        temp_data['id'] = data_all[idx]['id']

        meta_list.append([data_all[idx][el1] for el1 in processing_parameters.meta_fields])
        # try to load the motifs and latents
        try:
            latents = pd.read_hdf(el, 'latents')
            motifs = pd.read_hdf(el, 'motifs')
            egocentric_coords = pd.read_hdf(el, 'egocentric_coord')
            egocentric_coords = egocentric_coords.loc[:, ['cricket_0_x', 'cricket_0_y']]
            egocentric_coords = egocentric_coords.rename(columns={'cricket_0_x': 'ego_cricket_x',
                                                                  'cricket_0_y': 'ego_cricket_y'})
            # pad the latents
            [latents, motifs] = fl.pad_latents([latents, motifs], temp_data.shape[0])
            # concatenate with the main data
            temp_data = pd.concat([temp_data, egocentric_coords, latents, motifs], axis=1)
        except KeyError:
            print(f'No latents in file {el}')
        data_list.append(temp_data)
        frame_list.append([time_stamp, 0, temp_data.shape[0]])
    except KeyError:
        # data_list.append([])
        frame_list.append([time_stamp, 0, 0])

In [ ]:
%%time
# define the regression parameters and perform the regression
importlib.reload(class_fun)
# parameters
target_behavior = 'mouse_speed'
time_shift = 0
chunk = True
repeats = 10
shuffle_f = True
chunk_size = 0.05
chunk_size_shuffle = 0.05

# get the cell labels
labels = list(np.unique(np.array([el.columns for el in data_list]).flatten()))
cells = [el for el in labels if 'cell' in el]
# get the data
sub_data = [el[[target_behavior]+cells] for el in data_list if (target_behavior in el.columns)]
sub_data = pd.concat(sub_data)
# get the parameter of interest
parameter_working = sub_data.loc[:, target_behavior].to_numpy().copy()
calcium_data_working = np.array(sub_data[cells].copy())

# bin the parameter
# parameter_working = stat.binned_statistic(parameter_working, parameter_working, bins=5, statistic='count')[2]
bin_ranges = [np.percentile(parameter_working, el) for el in np.linspace(0, 100, 6)]
parameter_working = stat.binned_statistic(parameter_working, parameter_working, bins=bin_ranges, statistic='count')[2]

# allocate lists for the outputs
pred_list = []
coeff_list = []
score_list = []
# for all the repeats
for reps in np.arange(repeats):
    # create the regressor
#     regressor = lin.TweedieRegressor(alpha=0.01, max_iter=5000, fit_intercept=False, power=0)
#     regressor = AutoReg(lags=1)
#     regressor = lin.MultiTaskElasticNetCV(max_iter=5000, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], 
#                            n_jobs=7, alphas=[.0001, .01, .1, 1, 10, 100], fit_intercept=True)
#                            n_jobs=7, alphas=[.001])
#     regressor = lin.HuberRegressor(alpha=.1, max_iter=5000, fit_intercept=True, epsilon=10)
#     regressor = svm.SVR(max_iter=10000, kernel='rbf', C=100)
#     regressor = svm.LinearSVR(max_iter=5000, C=100)
#     regressor = svm.LinearSVC(max_iter=1000, C=1000, class_weight='balanced')
    regressor = svm.SVC(max_iter=10000, C=.1, class_weight='balanced', kernel='rbf')

    # run the training function
    linear_pred, coefficients, cc_score = class_fun.train_test_regressor(parameter_working,
                                                                         calcium_data_working,
                                                                         preprocessing.StandardScaler,
                                                                         regressor,
#                                                                          stat.spearmanr,
#                                                                          smet.r2_score,
                                                                         smet.balanced_accuracy_score,
                                                                         shuffle_f=shuffle_f,
                                                                         time_s=time_shift,
                                                                         shuffle=False,
                                                                         empty=False,
                                                                         chunk=chunk,
                                                                         chunk_size=chunk_size,
                                                                         chunk_size_shuffle=chunk_size_shuffle,
                                                                         test_size=0.3)

    pred_list.append(linear_pred)
    coeff_list.append(coefficients)
    score_list.append(cc_score)
print(f'Mean performance: {np.mean(score_list)}')
# print(f'Correlation of the mean prediction: {stat.spearmanr(parameter_working[time_shift:], np.mean(pred_list, axis=0))[0]}')


In [ ]:
freq, location = np.histogram(parameter_working)
hv.Scatter((location, freq)).opts(tools=['hover'])

In [ ]:
# print(np.array(pred_list)[:, trial_range].shape)
print(data_list[0].columns[:50])

# Regression example plot 

In [ ]:
# plot the predictions

importlib.reload(fp)
plot_list = []

# define the section of the trial to plot
trial_range = np.arange(500)
target_stretch = np.array(pred_list)[:, trial_range]
target_time = []
for idx, el in enumerate(data_list):
    if idx == 0:
        target_time.append(el['time_vector'].to_numpy())
    else:
        temp_time = el['time_vector'].to_numpy()
        target_time.append(temp_time + target_time[-1][-1])
            
#     target_time = pd.concat([el['time_vector'] for el in data_list], axis=0)

# x = np.arange(target_stretch.shape[1])
x = np.concatenate(target_time, axis=0)[trial_range]
# # for all the repeats
# for reps in np.arange(repeats):
#     plot = hv.Curve((x, pred_list[reps]))
#     plot.opts(width=1000)
#     plot_list.append(plot)
# also plot the sem for each time point
mean = np.mean(target_stretch, axis=0)
# sem = stat.sem(pred_list, axis=0)
std = np.std(target_stretch, axis=0)
plot = hv.Spread((x, mean, std)).opts(color='blue')
mean_plot = hv.Curve((x, mean))
mean_plot.opts(color='blue', line_dash='dotted')
plot_list.append(plot)
plot_list.append(mean_plot)
# and the original trace
# param_plot = parameter_working[~np.isnan(parameter_working)].copy()
param_plot = parameter_working.copy()
if time_shift >= 0:
    y = param_plot[time_shift:].copy()
else:
    y = param_plot[:time_shift].copy()
y = y[trial_range]

plot = hv.Curve((x, y))
# plot.opts(line_dash='dotted', title=f'Mean cc: {np.mean(score_list):0.2f}')
plot.opts(color='black')
plot_list.append(plot)
    
plot_overlay = hv.Overlay(plot_list)
plot_overlay.opts(opts.Curve(width=1000, height=300, xlabel='Time (s)', ylabel=' '.join((label_dict[target_behavior], units_dict[target_behavior]))))

# assemble the file name
save_name = os.path.join(save_path, '_'.join((target_document, 'Reg_ex')) + '.png')
# save the figure
fig = fp.save_figure(plot_overlay, save_name, fig_width=23, dpi=1200, fontsize=target_document, target='screen')


In [ ]:
# plot the weights

plot_list = []

x = np.arange(coeff_list[0].shape[0])
# for all the repeats
for reps in np.arange(repeats):
    plot = hv.Scatter((x, coeff_list[reps]))
    plot.opts(width=1000, size=10)
    plot_list.append(plot)
# also plot the sem for each weight
mean = np.mean(coeff_list, axis=0)
# sem = stat.sem(coeff_list, axis=0)
std = np.std(coeff_list, axis=0)
plot = hv.ErrorBars((x, mean, std))
plot.opts(title=f'Average weight std: {np.mean(std):0.2f}')
plot_list.append(plot)

hv.Overlay(plot_list)

In [ ]:
# Plot importance index

# plot_list = []

x = np.arange(coeff_list[0].shape[0])
# # for all the repeats
# for reps in np.arange(repeats):
#     plot = hv.Scatter((x, coeff_list[reps]))
#     plot.opts(width=1000, size=10)
#     plot_list.append(plot)

importance_list = []
# for all of the repeats
for reps in np.arange(repeats):
    importance_list.append(coeff_list[reps]/np.sum(coeff_list[reps]))
# also plot the sem for each weight
importance = np.sum(coeff_list, axis=0)
# sem = stat.sem(coeff_list, axis=0)
# std = np.std(coeff_list, axis=0)
# plot = hv.ErrorBars((x, mean, std))
plot = hv.Curve((x, importance))
plot.opts(title=f'Average weight std: {np.mean(std):0.2f}')
plot_list.append(plot)

hv.Overlay(plot_list)

In [ ]:
%%time
# calculate regression over time

importlib.reload(class_fun)
# parameters
target_behavior = 'cricket_0_x'
time_shift = [-40, -30, -20, -10, 0, 10, 20, 30, 40]
# time_shift = [-10, 0, 10]
# time_shift = [-15, -10, -5, 0, 5, 10, 15]
chunk = True
repeats = 100
# shuffle = True
chunk_size = 0.05
chunk_size_shuffle = 0.05

# get the cell labels
labels = list(np.unique(np.array([el.columns for el in data_list]).flatten()))
cells = [el for el in labels if 'cell' in el]
# get the data
sub_data = [el[[target_behavior]+cells] for el in data_list if (target_behavior in el.columns)]
sub_data = pd.concat(sub_data)
# get the parameter of interest
parameter_working = sub_data.loc[:, target_behavior].to_numpy().copy()
calcium_data_working = np.array(sub_data[cells].copy())

# # bin the parameter
# parameter_working = stat.binned_statistic(parameter_working, parameter_working, bins=10, statistic='count')[2]

# allocate memory for the shuffles
pred_shuffle = []
coeff_shuffle = []
score_shuffle = []
# for real and shuffle
for realvshuffle in np.arange(2):
    
    if realvshuffle == 0:
        shuffle_f = False
    else:
        shuffle_f = True
    # allocate lists for the outputs
    pred_time = []
    coeff_time = []
    score_time = []
    # for all the time shifts
    for time_s in time_shift:
        pred_list = []
        coeff_list = []
        score_list = []
        # for all the repeats
        for reps in np.arange(repeats):
            # create the regressor
            regressor = lin.TweedieRegressor(alpha=0.01, max_iter=5000, fit_intercept=False, power=0)
#             regressor = lin.MultiTaskElasticNetCV(max_iter=5000, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], 
#                                    n_jobs=7, alphas=[.0001, .01, .1, 1, 10, 100], fit_intercept=True)
#                                    n_jobs=7, alphas=[.001])
#             regressor = lin.HuberRegressor(alpha=.1, max_iter=5000, fit_intercept=True, epsilon=10)
#             regressor = svm.SVR(max_iter=10000, kernel='rbf', C=100)
#             regressor = svm.LinearSVR(max_iter=5000, C=100)
#             regressor = svm.LinearSVC(max_iter=1000, C=1000, class_weight='balanced')
#             regressor = svm.SVC(max_iter=1000, C=.1, class_weight='balanced', kernel='rbf')
#             regressor = multi.OutputCodeClassifier(estimator=regressor, code_size=1, n_jobs=-1)

            # run the training function
            linear_pred, coefficients, cc_score = class_fun.train_test_regressor(parameter_working,
                                                                                 calcium_data_working,
                                                                                 preprocessing.StandardScaler,
                                                                                 regressor,
                                                                                 stat.spearmanr,
#                                                                                  smet.r2_score,
#                                                                                  smet.accuracy_score,
                                                                                 time_s=time_s,
                                                                                 shuffle_f=shuffle_f,
                                                                                 empty=False,
                                                                                 chunk=chunk,
                                                                                 test_size=0.3,
                                                                                 shuffle=False,
                                                                                 chunk_size=chunk_size,
                                                                                 chunk_size_shuffle=chunk_size_shuffle)
                                                                                 
            pred_list.append(linear_pred)
            coeff_list.append(coefficients)
            score_list.append(cc_score)
        pred_time.append([np.mean(pred_list, axis=0), np.std(pred_list, axis=0)])
        coeff_time.append([np.mean(coeff_list, axis=0), np.std(coeff_list, axis=0)])
        score_time.append([np.mean(score_list), np.std(score_list)])
#         # get the score
#         param_plot = parameter_working[~np.isnan(parameter_working)].copy()
#         if time_s >= 0:
#             param_plot = param_plot[time_s:].copy()
#         else:
#             param_plot = param_plot[:time_s].copy()
#         score_of_mean = stat.spearmanr(param_plot, pred_time[-1][0])[0]
#         score_of_std = stat.spearmanr(param_plot, pred_time[-1][1])[0]
#         score_time.append([score_of_mean, score_of_std])
    pred_shuffle.append(pred_time)
    coeff_shuffle.append(coeff_time)
    score_shuffle.append(score_time)
print(np.mean(score_list))

In [ ]:
# plot the results over time

plot_list = []
mean = hv.Scatter((time_shift, [el[0] for el in score_shuffle[0]]), kdims='Time shift', vdims='Activity')
std = hv.Spread((time_shift, [el[0] for el in score_shuffle[0]], [el[1] for el in score_shuffle[0]]))
mean.opts(width=1000, color='red', title=target_behavior)
std.opts(width=1000, color='red')
plot_list.append(mean*std)
mean = hv.Scatter((time_shift, [el[0] for el in score_shuffle[1]]))
std = hv.Spread((time_shift, [el[0] for el in score_shuffle[1]], [el[1] for el in score_shuffle[1]]))
mean.opts(width=1000, color='black')
std.opts(width=1000, color='black')
plot_list.append(mean*std)
hv.Overlay(plot_list)


In [ ]:
# compute the autocorrelation of a given variable

plot_list = []
autocorrelation = signal.correlate(parameter_working, parameter_working, 'same')
x = np.arange(-autocorrelation.shape[0]/2, autocorrelation.shape[0]/2)
plot = hv.Curve((x, autocorrelation))
plot.opts(width=1000, color='red')
plot_list.append(plot)
for reps in np.arange(repeats):
    parameter_shuffle = class_fun.chunk_shuffle(parameter_working, chunk_size_shuffle=0.01)
#     print(parameter_shuffle)
    auto_shuffle = signal.correlate(parameter_shuffle, parameter_shuffle, 'same')
    plot = hv.Curve((x, auto_shuffle))
    plot.opts(width=1000, color='black')
    plot_list.append(plot)

hv.Overlay(plot_list)